# Data Type Corrections for Medical Dataset

This notebook implements Task 1 of the assignment, focusing on examining and correcting data types in the medical dataset.

## Import Required Libraries

In [21]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
from pandas.api.types import CategoricalDtype

warnings.filterwarnings("ignore")

## Load and Display Initial Data

In [22]:
# Load the dataset
df = pd.read_excel("C:\\Users\\galle\\Documents\\www\\qut\\assignment1\\medical-dataset.xlsx")

print("--- Initial Data Types ---")
df.dtypes

--- Initial Data Types ---


survey_date                object
region                     object
country                    object
ip_latitude               float64
ip_longitude              float64
gender                     object
age                        object
height                      int64
weight                      int64
bmi                       float64
blood_type                 object
insurance                  object
income                     object
smoking                    object
alcohol                   float64
cocaine                   float64
contacts_count            float64
public_transport_count    float64
working                    object
worried                   float64
covid19_positive            int64
covid19_symptoms            int64
covid19_contact             int64
asthma                      int64
kidney_disease              int64
liver_disease               int64
compromised_immune          int64
heart_disease               int64
lung_disease                int64
diabetes      

## Task 1: Data Type Corrections

We'll now implement the necessary data type corrections for each variable in the dataset.

### 1. Date Conversion

In [23]:
def convert_date(date_str):
    if pd.isna(date_str):
        return pd.NaT
    try:
        for fmt in ['%Y-%m-%d %H:%M:%S', '%d/%m/%Y', '%Y-%m-%d']:
            try:
                return pd.to_datetime(date_str, format=fmt)
            except:
                continue
        return pd.to_datetime(date_str, infer_datetime_format=True)
    except:
        return pd.NaT

df['survey_date'] = df['survey_date'].apply(convert_date)

### 2. Age - Convert to Ordered Categorical

In [24]:
df['age'] = df['age'].replace("100_110", "90_plus")
age_order = ['0_10', '10_20', '20_30', '30_40', '40_50', '50_60', 
             '60_70', '70_80', '80_90', '90_plus']
age_dtype = pd.api.types.CategoricalDtype(categories=age_order, ordered=True)
df['age'] = df['age'].astype(age_dtype)

# Display age categories and their counts
print("Age Categories:")
print(df['age'].cat.categories)
print("\nAge Value Counts:")
df['age'].value_counts(sort=False)

Age Categories:
Index(['0_10', '10_20', '20_30', '30_40', '40_50', '50_60', '60_70', '70_80',
       '80_90', '90_plus'],
      dtype='object')

Age Value Counts:


age
0_10         14
10_20       205
20_30       996
30_40      1412
40_50      1111
50_60       926
60_70       765
70_80       354
80_90         0
90_plus       6
Name: count, dtype: int64

### 3. Gender - Convert to Nominal Categorical

In [25]:
df['gender'] = df['gender'].astype('category')
print("Gender Categories:")
df['gender'].value_counts()

Gender Categories:


gender
male      2985
female    2790
other       14
Name: count, dtype: int64

### 4. Region and Country - Convert to Categorical with 'Unknown'

In [26]:
for col in ['region', 'country']:
    df[col] = df[col].astype('category')
    if 'Unknown' not in df[col].cat.categories:
        df[col] = df[col].cat.add_categories('Unknown')
    df[col] = df[col].fillna('Unknown')
    
    print(f"\n{col.capitalize()} Categories:")
    print(df[col].value_counts())


Region Categories:
region
Unknown    4227
EU          801
SA          399
AS          206
AF           79
OC           77
Name: count, dtype: int64

Country Categories:
country
US    3885
GB     365
BR     270
CA     208
MX      94
      ... 
BS       1
BM       1
AZ       1
AW       1
JE       1
Name: count, Length: 95, dtype: int64


### 5. Height and Weight - Convert to Integer

In [27]:
df['height'] = df['height'].astype('Int64')  # Using Int64 to handle NaN values
df['weight'] = df['weight'].astype('Int64')

print("Height Summary:")
print(df['height'].describe())
print("\nWeight Summary:")
print(df['weight'].describe())

Height Summary:
count        5789.0
mean     172.030057
std         10.9176
min           110.0
25%           164.0
50%           172.0
75%           180.0
max           238.0
Name: height, dtype: Float64

Weight Summary:
count       5789.0
mean     84.773881
std        23.5763
min           44.0
25%           68.0
50%           82.0
75%           96.0
max          180.0
Name: weight, dtype: Float64


### 6. BMI - Keep as Float

In [28]:
df['bmi'] = df['bmi'].astype('float64')
print("BMI Summary:")
df['bmi'].describe()

BMI Summary:


count    5789.000000
mean       28.570116
std         7.674528
min        11.900000
25%        23.500000
50%        26.800000
75%        31.800000
max       125.000000
Name: bmi, dtype: float64

### 7. Blood Type - Convert to Categorical with 'Unknown'

In [29]:
df['blood_type'] = df['blood_type'].astype('category')
if 'Unknown' not in df['blood_type'].cat.categories:
    df['blood_type'] = df['blood_type'].cat.add_categories('Unknown')
df['blood_type'] = df['blood_type'].fillna('Unknown')

print("Blood Type Categories:")
df['blood_type'].value_counts()

Blood Type Categories:


blood_type
unknown    1784
ap         1279
op         1251
on          417
bp          394
an          244
abp         191
Unknown     100
bn           91
abn          38
Name: count, dtype: int64

### 8. Insurance - Convert to Boolean with NA for Unknown

In [30]:
df['insurance'] = df['insurance'].map({'Yes': True, 'No': False, 'Unknown': pd.NA})
print("Insurance Value Counts:")
df['insurance'].value_counts()

Insurance Value Counts:


Series([], Name: count, dtype: int64)

### 9. Income - Convert to Categorical

In [31]:
df['income'] = df['income'].astype('category')
print("Income Categories:")
df['income'].value_counts()

Income Categories:


income
med      2142
high     1886
low       398
blank     122
gov        88
Name: count, dtype: int64

### 10. Smoking - Convert to Categorical with Bins

In [32]:
smoking_bins = {
    'quit0': 'Former Smoker',
    'quit5': 'Former Smoker',
    'quit10': 'Former Smoker',
    'vape': 'Vape User',
    'yesheavy': 'Heavy Smoker',
    'light': 'Light Smoker',
    'medium': 'Medium Smoker',
    'never': 'Never Smoked',
    -1: 'Unknown'
}
df['smoking'] = df['smoking'].replace(smoking_bins)
df['smoking'] = df['smoking'].astype('category')

print("Smoking Categories:")
df['smoking'].value_counts()

Smoking Categories:


smoking
Never Smoked     3713
Former Smoker    1277
Vape User         247
yesmedium         230
yeslight          206
Heavy Smoker       80
Name: count, dtype: int64

### 11. Alcohol - Convert to Categorical Ordinal

In [33]:
df['alcohol'] = df['alcohol'].replace(-1, 'Unknown')
df['alcohol'] = df['alcohol'].astype('category')

print("Alcohol Categories:")
df['alcohol'].value_counts()

Alcohol Categories:


alcohol
Unknown    1920
0.0        1275
1.0         612
2.0         466
3.0         310
4.0         247
5.0         194
7.0         166
14.0        117
10.0        108
6.0          86
13.0         72
12.0         64
8.0          48
9.0          39
11.0         27
Name: count, dtype: int64

### 12. Cocaine - Convert to Categorical with Unknown

In [34]:
df['cocaine'] = df['cocaine'].replace(-1, 'Unknown')
df['cocaine'] = df['cocaine'].astype('category')

print("Cocaine Categories:")
df['cocaine'].value_counts()

Cocaine Categories:


cocaine
Unknown    1078
0.0         228
1.0          33
2.0          17
5.0          14
4.0          12
3.0          12
7.0           7
9.0           5
28.0          4
10.0          3
20.0          3
27.0          3
6.0           2
16.0          2
23.0          2
25.0          1
15.0          1
22.0          1
21.0          1
14.0          1
Name: count, dtype: int64

### 13. Contact Counts - Convert to Int64

In [35]:
df['contacts_count'] = df['contacts_count'].astype('Int64')
df['public_transport_count'] = df['public_transport_count'].astype('Int64')

print("Contacts Count Summary:")
print(df['contacts_count'].describe())
print("\nPublic Transport Count Summary:")
print(df['public_transport_count'].describe())

Contacts Count Summary:
count      5747.0
mean     7.828432
std      6.672943
min           0.0
25%           3.0
50%           5.0
75%          10.0
max          21.0
Name: contacts_count, dtype: Float64

Public Transport Count Summary:
count      4572.0
mean     0.415573
std      1.811049
min           0.0
25%           0.0
50%           0.0
75%           0.0
max          15.0
Name: public_transport_count, dtype: Float64


### 14. Working - Convert to Categorical

In [36]:
df['working'] = df['working'].astype('category')
print("Working Status Categories:")
df['working'].value_counts()

Working Status Categories:


working
stopped                1732
never                  1577
travel critical        1312
travel non critical     620
home                    506
Name: count, dtype: int64

### 15. Worried - Convert to Integer

In [37]:
df['worried'] = df['worried'].astype('Int64')
print("Worried Level Summary:")
df['worried'].describe()

Worried Level Summary:


count      4518.0
mean     3.530987
std      0.792812
min           1.0
25%           3.0
50%           4.0
75%           4.0
max           5.0
Name: worried, dtype: Float64

### 16. Health Conditions - Convert to Boolean

In [38]:
health_conditions = [
    'covid19_positive', 'covid19_symptoms', 'covid19_contact',
    'asthma', 'kidney_disease', 'liver_disease', 'compromised_immune',
    'heart_disease', 'lung_disease', 'diabetes', 'hiv_positive',
    'other_chronic', 'nursing_home', 'health_worker'
]

for col in health_conditions:
    df[col] = df[col].astype('bool')
    
print("Health Conditions Summary:")
for col in health_conditions:
    print(f"\n{col}:")
    print(df[col].value_counts())

Health Conditions Summary:

covid19_positive:
covid19_positive
False    3454
True     2335
Name: count, dtype: int64

covid19_symptoms:
covid19_symptoms
False    4792
True      997
Name: count, dtype: int64

covid19_contact:
covid19_contact
False    4800
True      989
Name: count, dtype: int64

asthma:
asthma
False    5032
True      757
Name: count, dtype: int64

kidney_disease:
kidney_disease
False    5726
True       63
Name: count, dtype: int64

liver_disease:
liver_disease
False    5745
True       44
Name: count, dtype: int64

compromised_immune:
compromised_immune
False    5444
True      345
Name: count, dtype: int64

heart_disease:
heart_disease
False    5607
True      182
Name: count, dtype: int64

lung_disease:
lung_disease
False    5654
True      135
Name: count, dtype: int64

diabetes:
diabetes
False    5320
True      469
Name: count, dtype: int64

hiv_positive:
hiv_positive
False    5766
True       23
Name: count, dtype: int64

other_chronic:
other_chronic
False    5411
True 

### 17. Risk Values - Convert to Float and Normalize

In [39]:
df['risk_infection'] = df['risk_infection'].astype('float64') / 100
df['risk_mortality'] = df['risk_mortality'].astype('float64') / 100

print("Risk Infection Summary:")
print(df['risk_infection'].describe())
print("\nRisk Mortality Summary:")
print(df['risk_mortality'].describe())

Risk Infection Summary:
count    5789.000000
mean        0.509772
std         0.440276
min         0.050000
25%         0.050000
50%         0.370000
75%         1.000000
max         1.000000
Name: risk_infection, dtype: float64

Risk Mortality Summary:
count    5789.000000
mean        0.005623
std         0.010778
min         0.000500
25%         0.000500
50%         0.001560
75%         0.005990
max         0.286980
Name: risk_mortality, dtype: float64


## Final Data Types Check

In [40]:
print("Final Data Types:")
df.dtypes

Final Data Types:


survey_date               datetime64[ns]
region                          category
country                         category
ip_latitude                      float64
ip_longitude                     float64
gender                          category
age                             category
height                             Int64
weight                             Int64
bmi                              float64
blood_type                      category
insurance                         object
income                          category
smoking                         category
alcohol                         category
cocaine                         category
contacts_count                     Int64
public_transport_count             Int64
working                         category
worried                            Int64
covid19_positive                    bool
covid19_symptoms                    bool
covid19_contact                     bool
asthma                              bool
kidney_disease  